<a href="https://colab.research.google.com/github/andreaFernandezMa/diplomado_de/blob/main/Tareas_Andrea_Fernandez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pyspark Install

In [ ]:
# descomentar esta linea si falla al instalar openjdk
!apt-get update

Ign:1 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [92.1 kB]
Hit:14 http://ppa.lau

In [ ]:
#Instalacion jdk y descargando spark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

In [ ]:
#Instalación findspark
!pip install -q findspark
#Seteo de variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

In [ ]:
#Instalar paquete pyspark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 49 kB/s 
     |████████████████████████████████| 199 kB 47.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=f286c831edc2a2f6f9e9b04ee1e6bcf89033aafebeacb9c374bfaf681a61d015
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


#Crear SparkSession

In [ ]:
#Iniciar SparkSession
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
#Otras Librerias
from pyspark.sql.functions import *


In [ ]:
#Conectar google drive con Colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# TAREA 1

## Parte 1 (30%)

Utilizando el dataset de [Walmart Stock](https://drive.google.com/file/d/154-RydBa4MOaubvLsiiGtc2HJW1alCkv/view?usp=sharing) realice lo siguiente:

*   Inicie una sesión de Spark
*   Cargue el archivo de Walmart Stock csv desde su cuenta Drive a un dataframe, permita que spark infiera el schema y tipos de datos
*   imprima el schema, si no es correcto, cree usted el schema y cargue de nuevo el archivo.
*   Imprima las primeras 5 filas
*   Relice un *Describe* de la data
*   Cree un nuevo Dataframe con una columna llamada **HV Ratio** el cual tenga la tasa entre **High** versus **Volume** 
*   Determine la media de la columna **Close**
*   Determine el máximo valor de **High** por año.
*   Determine el promedio de **Close** por cada mes.

####Inicie una sesión de Spark

In [ ]:
#Inicie una sesión de Spark
#comprobar que la sesion de spark esta ok
spark

####Cargue el archivo de Walmart Stock csv

Se creo el schema debido a que los datos eraan leidos como string

In [202]:
from pyspark.sql.types import (
    DateType,
    FloatType,
    IntegerType,
    StructType,
    StructField, 
    StringType, 
    IntegerType,
    LongType,
    DecimalType,
    BooleanType
)

In [ ]:
#Creacion de schema
walmart_schema = StructType(
    [
        StructField('Date', DateType(), False),
        StructField('Open',FloatType(), False),
        StructField('High',FloatType(), False),
        StructField('Low',FloatType(), False),
        StructField('Close',FloatType(), False),
        StructField('Volume',IntegerType(), False),
        StructField('Adj Close',FloatType(), False),
   ]
)

In [ ]:
#Cargue el archivo de Walmart Stock csv desde su cuenta Drive a un dataframe, permita que spark infiera el schema y tipos de datos
walmart = '/content/tarea1.1/walmart_stock.csv'
#df_walmart = spark.read.csv(walmart, header=True, inferSchema=True) 
df_walmart = spark.read.csv(walmart, header=True, schema=walmart_schema) 

####imprima el schema
Se importaron los formatos que necesitaremos para poder desarrollar el ejercicio

In [ ]:
#imprima el schema, si no es correcto, cree usted el schema y cargue de nuevo el archivo.
df_walmart.printSchema() 

root
 |-- Date: date (nullable = true)
 |-- Open: float (nullable = true)
 |-- High: float (nullable = true)
 |-- Low: float (nullable = true)
 |-- Close: float (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: float (nullable = true)



####Imprima las primeras 5 filas

In [ ]:
#Imprima las primeras 5 filas
df_walmart.show(5)

+----------+-----+-----+-----+-----+--------+---------+
|      Date| Open| High|  Low|Close|  Volume|Adj Close|
+----------+-----+-----+-----+-----+--------+---------+
|2012-01-03|59.97|61.06|59.87|60.33|12668800|52.619236|
|2012-01-04|60.21|60.35|59.47|59.71| 9593300|52.078476|
|2012-01-05|59.35|59.62|58.37|59.42|12768200| 51.82554|
|2012-01-06|59.42|59.45|58.87| 59.0| 8069400| 51.45922|
|2012-01-09|59.03|59.55|58.92|59.18| 6679300|51.616215|
+----------+-----+-----+-----+-----+--------+---------+
only showing top 5 rows



####Relice un Describe de la data

In [ ]:
# Relice un Describe de la data
df_walmart.describe()

DataFrame[summary: string, Open: string, High: string, Low: string, Close: string, Volume: string, Adj Close: string]

####Cree un nuevo Dataframe 
Con una columna llamada HV Ratio el cual tenga la tasa entre High versus Volume

In [ ]:
#Cree un nuevo Dataframe con una columna llamada HV Ratio el cual tenga la tasa entre High versus Volume
df_walmart2 = df_walmart.withColumn('HV', col('High')/col('Volume'))

In [ ]:
df_walmart2.show()

+----------+-----+-----+-----+-----+--------+---------+--------------------+
|      Date| Open| High|  Low|Close|  Volume|Adj Close|                  HV|
+----------+-----+-----+-----+-----+--------+---------+--------------------+
|2012-01-03|59.97|61.06|59.87|60.33|12668800|52.619236|4.819714682786927E-6|
|2012-01-04|60.21|60.35|59.47|59.71| 9593300|52.078476|6.290848662516662E-6|
|2012-01-05|59.35|59.62|58.37|59.42|12768200| 51.82554| 4.66941298944916E-6|
|2012-01-06|59.42|59.45|58.87| 59.0| 8069400| 51.45922| 7.36733843444859E-6|
|2012-01-09|59.03|59.55|58.92|59.18| 6679300|51.616215|8.915604814435727E-6|
|2012-01-10|59.43|59.71|58.98|59.04| 6907300| 51.49411|8.644477449144044E-6|
|2012-01-11|59.06|59.53|59.04| 59.4| 6365600|51.808098|9.351828386844425E-6|
|2012-01-12|59.79| 60.0| 59.4| 59.5| 7236400|51.895317| 8.29141562102703E-6|
|2012-01-13|59.18|59.61|59.01|59.54| 7729300|51.930202|7.712212051589609E-6|
|2012-01-17|59.87|60.11|59.52|59.85| 8500000| 52.20058|7.071764777688419...|

####Determine la media de la columna Close

In [ ]:
#Determine la media de la columna Close
df_walmart.select(avg('Close').alias('promedio_close')).show()

+-----------------+
|   promedio_close|
+-----------------+
|72.38844997363553|
+-----------------+



####Determine el máximo valor de High por año

In [ ]:
df_walmart2.groupBy(year(col('Date')).alias('Año')).max('High').show()

+----+---------+
| Año|max(High)|
+----+---------+
|2015|    90.97|
|2013|    81.37|
|2014|    88.09|
|2012|     77.6|
|2016|    75.19|
+----+---------+



####Determine el promedio de Close por cada mes.

In [ ]:
df_walmart2.groupBy(month(col('Date')).alias('Mes')).avg('Close').show()

+---+-----------------+
|Mes|       avg(Close)|
+---+-----------------+
| 12|72.84792482628012|
|  1| 71.4480196131338|
|  6| 72.4953774506191|
|  3|71.77794376266337|
|  5|72.30971685445533|
|  9|72.18411782208611|
|  4|72.97361900692894|
|  8| 73.0298185521906|
|  7|74.43971944078106|
| 10| 71.5785454489968|
| 11|72.11108927207418|
|  2|71.30680438169499|
+---+-----------------+



## Parte 2 (70%)

Se tiene data de logs en JSON comprimidos que se almacenan cada una hora, se pretende enviar los logs de la hora anterior a la actual a una API, pero es necesario realizar un procesamiento de ellos primero. Los campos de los logs se definen de la siguiente manera:

- `index_name`: correponde a un `string`el cual es un indexador del log y no puede ser `null`.
- `detail`: es un diccionario tipo JSON, su estructura es dinámica y puede variar según el log. No puede ser `null`.
- `timestamp`: corresponde al timestamp en unix epoch del instante en que fué generado el log. Debe ser un entero, pero no siempre llega así, tampoco puede ser `null`.
- `group_by`: campo opcional, el cual corresponde a un string que sirve para agrupar el log.
- `tags`: campo opcional, correponde a un array de strings para permitir hacer indexar los logs en la API y realizar queries más eficientes.

Usted debe procesar los logs en un notebook de python con PySpark y escribir la data resultante de manera que cumpla con lo siguiente:

- El archivo final debe contar con `index_name`, `detail`, `timestamp` y `tags`.
- En caso de que exista `group_by` para un log, debe ser añadido al array de `tags`.
- Si un log no tiene `tags` o es `null`, debe retornar un array vacío.
- Asegurarse de que el campo `timestamp` sea de tipo entero. (Hint: cast y round)
- Debe crear un schema para procesar la data, si bien con grandes volumenes de datos se infiere bien, para pocos logs puede no inferir correctamente.
- Debe guardar la data procesada como archivo tipo JSON particionada en su Google Drive.
- La data debe estar particionada por:
  - `env`: corresponde al ambiente de ejecución (prod y qa)
  - `index`: correpsonde al mismo valor de `index_name`.
  - `group_by`: valor que ya viene en `group_by`, en caso de ser null, asignar valor: `NO_GROUP`
  - `Year`, `Month`, `Day`, `Hour`, donde cada una de esas columnas corresponde a la hora de los logs en el path (hora anterior)

Ejemplo ruta de salida:
`drive/MyDrive/.../env=prod/index=dummy_index/group_by=NO_GROUP/year=2022/month=08/day=17/hour=21/filename.json`

Ejemplo archivo de salida:
```
{
  "index_name" : "dummy_name",
  "detail" : {#some dummy dict}
  "timestamp": 1231231321
  "tags" : ["tag_0",....,"tag_n"]
}
```

Consideraciones:
- Asuma que la data que se encuentra en el path de Googe Drive es de la hora pasada a la actual.
- En la creación de su Schema considere que algunos campos no siempre tendrán la misma estructura, para ello simplemente tratelos como `String`.
- Para el ambiente, simplemente realice la ejecución 2 veces, una con `qa` y otra con `prod`.
- Note que para particionar por fecha debe tomar la hora actual y restarle una hora (libería datetime en python).

####Unir archivos en 1 dataframe
Una vez unida la data podemos tener visualizacion, esto nos ayudará a saber si se debe hacer algun cambio

In [ ]:
input_path_order_data = '/content/tarea1.2'
order_df = spark.read.json(input_path_order_data)

In [ ]:
order_df.printSchema() 

root
 |-- detail: struct (nullable = true)
 |    |-- date: string (nullable = true)
 |    |-- item_uuid: string (nullable = true)
 |    |-- last_date_check: string (nullable = true)
 |    |-- number_of_products: long (nullable = true)
 |    |-- order_uuid: string (nullable = true)
 |    |-- shopper_id: long (nullable = true)
 |    |-- status: string (nullable = true)
 |    |-- store_id: long (nullable = true)
 |-- group_by: string (nullable = true)
 |-- index_name: string (nullable = true)
 |-- tags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- timestamp: double (nullable = true)



In [ ]:
order_df.show(5)

+--------------------+----------+-------------------+------------+------------+
|              detail|  group_by|         index_name|        tags|   timestamp|
+--------------------+----------+-------------------+------------+------------+
|[2022/08/16,,,, f...|Dispatched|shoppers_dispatcher|[tag3, tag3]|1.66079041E9|
|[2022/08/05,,,, f...| Delivered|shoppers_dispatcher|[tag3, tag2]|1.66079041E9|
|[2022/08/16,,,, f...| Scheduled|shoppers_dispatcher|[tag1, tag2]|1.66079041E9|
|[2022/08/03,,,, 2...| Scheduled|shoppers_dispatcher|[tag3, tag0]|1.66079041E9|
|[2022/08/04,,,, 4...| Delivered|shoppers_dispatcher|        null|1.66079041E9|
+--------------------+----------+-------------------+------------+------------+
only showing top 5 rows



####Esquema data

In [ ]:
order_schema = StructType(
    [
        StructField('index_name', StringType(), False),
        StructField('detail',StringType(), False),
        StructField('timestamp',IntegerType(), False),
        StructField('tags',StringType(), True),
        StructField('group_by', StringType(), True),
   ]
)

In [ ]:
order_df2 = spark.read.json(input_path_order_data,schema=order_schema)

In [ ]:
order_df2.show(5)

+-------------------+--------------------+----------+---------------+----------+
|         index_name|              detail| timestamp|           tags|  group_by|
+-------------------+--------------------+----------+---------------+----------+
|shoppers_dispatcher|{"shopper_id":689...|1660790410|["tag3","tag3"]|Dispatched|
|shoppers_dispatcher|{"shopper_id":353...|1660790410|["tag3","tag2"]| Delivered|
|shoppers_dispatcher|{"shopper_id":699...|1660790410|["tag1","tag2"]| Scheduled|
|shoppers_dispatcher|{"shopper_id":210...|1660790410|["tag3","tag0"]| Scheduled|
|shoppers_dispatcher|{"shopper_id":657...|1660790410|           null| Delivered|
+-------------------+--------------------+----------+---------------+----------+
only showing top 5 rows



In [ ]:
#Para reemplazar valor
from pyspark.sql.functions import regexp_replace

In [ ]:
order_formato_salida = order_df2.select('index_name','detail','timestamp','tags','group_by')

####Preparar data para guardar

In [ ]:
#importar libreria para poder trabajar con fechas
from datetime import datetime, timedelta

In [ ]:
current_time = datetime.utcnow()
#hora anterior
last_hour = current_time - timedelta(hours=1)
env_prod = 'prod'
env_qa = 'qa'

In [ ]:
order_formato_salida_prod = (
    order_formato_salida.withColumn('env', lit(env_prod))
    .withColumn('year', lit(last_hour.year))
    .withColumn('month', lit(last_hour.month))
    .withColumn('day', lit(last_hour.day))
    .withColumn('hour', lit(last_hour.hour))
)
order_formato_salida_qa = (
    order_formato_salida.withColumn('env', lit(env_qa))
    .withColumn('year', lit(last_hour.year))
    .withColumn('month', lit(last_hour.month))
    .withColumn('day', lit(last_hour.day))
    .withColumn('hour', lit(last_hour.hour))
)

####Particionar data

In [ ]:
#Donde seran guardadas las particiones
output_path_prod = '/content/drive/MyDrive/M6/output_path/prod'
output_path_qa = '/content/drive/MyDrive/M6/output_path/qa'

In [ ]:
#guardar data prod particionada 
order_formato_salida_prod.write.partitionBy(
    'env','index_name', 'group_by', 'year', 'month', 'day', 'hour').mode('append').json(output_path_prod)  

In [ ]:
#guardar data prod qa 
order_formato_salida_qa.write.partitionBy(
    'env','index_name', 'group_by', 'year', 'month', 'day', 'hour').mode('append').json(output_path_qa)  

# TAREA 2

##Inicio, cargar dato y asignar esquemas

###Cargar datos

In [198]:
dog_food = '/content/tarea2/dog_food.csv'
hack_data = '/content/tarea2/hack_data.csv'

###Creacion Esquema

In [233]:
#Creacion de schema food_schema
dog_food_schema = StructType(
    [
        StructField('A',IntegerType(), True),
        StructField('B',IntegerType(), True),
        StructField('C',DecimalType(10,1), True),
        StructField('D',IntegerType(), True),
        StructField('Spoiled',DecimalType(10,1), True),
   ]
)

#Creacion de schema data_schema
hack_data_schema = StructType(
    [
        StructField('Session_Connection_Time', DecimalType(10,2), True),
        StructField('Bytes Transferred',DecimalType(10,2), True),
        StructField('Kali_Trace_Used', IntegerType(), True),
        StructField('Servers_Corrupted',DecimalType(10,2), True),
        StructField('Pages_Corrupted',DecimalType(10,2), True),
        StructField('Location', StringType(), True),
        StructField('WPM_Typing_Speed',DecimalType(10,2), True),
   ]
)

In [234]:
#Asignar datas a data frame con esquema
df_dog_food = spark.read.csv(dog_food, header=True, schema=dog_food_schema) 
df_hack_data = spark.read.csv(hack_data, header=True, schema=hack_data_schema)  

In [250]:
#Verificando que no existan valores null y saber tipos de datos en df_dog_food
df_dog_food.groupBy(col('A'),col('B'),col('C'),col('D'),col('Spoiled')).count().show()

+---+---+----+---+-------+-----+
|  A|  B|   C|  D|Spoiled|count|
+---+---+----+---+-------+-----+
|  2|  4|13.0|  4|    1.0|    1|
|  6|  2|12.0|  6|    1.0|    1|
|  8| 10| 9.0|  8|    0.0|    1|
| 10|  9| 7.0|  4|    0.0|    1|
|  2|  6|12.0|  2|    1.0|    1|
|  4|  3| 9.0|  6|    0.0|    1|
|  4|  2|12.0|  1|    1.0|    1|
|  4|  9|12.0|  8|    1.0|    1|
|  1|  2| 9.0|  1|    0.0|    1|
|  7|  7|11.0|  4|    1.0|    1|
|  4|  3| 8.0|  8|    0.0|    1|
|  6|  8| 8.0|  9|    0.0|    1|
|  8|  6| 8.0|  2|    0.0|    1|
|  2|  2| 8.0|  1|    0.0|    2|
|  4|  4| 9.0|  1|    0.0|    1|
|  9|  4|12.0|  3|    1.0|    1|
|  8| 10|13.0|  3|    1.0|    1|
|  9|  2| 9.0|  9|    0.0|    1|
|  2|  3| 6.0|  9|    0.0|    1|
|  1|  6| 8.0|  3|    0.0|    1|
+---+---+----+---+-------+-----+
only showing top 20 rows



In [253]:
#Verificando que no existan valores null y saber tipos de datos en df_hack_data
df_hack_data.groupBy(col('Session_Connection_Time'), col('Bytes Transferred'),col('Kali_Trace_Used'),
                     col('Servers_Corrupted'),col('Pages_Corrupted'),col('Location'),col('WPM_Typing_Speed')
).count().show(10)

+-----------------------+-----------------+---------------+-----------------+---------------+--------------+----------------+-----+
|Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|      Location|WPM_Typing_Speed|count|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------+----------------+-----+
|                  15.00|           548.83|              1|             3.78|           7.00|       Bahamas|           68.53|    1|
|                  12.00|           424.83|              1|             2.53|           8.00|    Bangladesh|           69.99|    1|
|                  39.00|           861.74|              1|             6.82|          14.00|United Kingdom|           41.61|    1|
|                  17.00|           602.46|              1|             4.87|           9.00|         Sudan|           70.26|    1|
|                  18.00|           393.15|              0|             2.25

###Visualizacion de la data

In [254]:
df_dog_food.show()

+---+---+----+---+-------+
|  A|  B|   C|  D|Spoiled|
+---+---+----+---+-------+
|  4|  2|12.0|  3|    1.0|
|  5|  6|12.0|  7|    1.0|
|  6|  2|13.0|  6|    1.0|
|  4|  2|12.0|  1|    1.0|
|  4|  2|12.0|  3|    1.0|
| 10|  3|13.0|  9|    1.0|
|  8|  5|14.0|  5|    1.0|
|  5|  8|12.0|  8|    1.0|
|  6|  5|12.0|  9|    1.0|
|  3|  3|12.0|  1|    1.0|
|  9|  8|11.0|  3|    1.0|
|  1| 10|12.0|  3|    1.0|
|  1|  5|13.0| 10|    1.0|
|  2| 10|12.0|  6|    1.0|
|  1| 10|11.0|  4|    1.0|
|  5|  3|12.0|  2|    1.0|
|  4|  9|11.0|  8|    1.0|
|  5|  1|11.0|  1|    1.0|
|  4|  9|12.0| 10|    1.0|
|  5|  8|10.0|  9|    1.0|
+---+---+----+---+-------+
only showing top 20 rows



In [255]:
df_dog_food.printSchema()

root
 |-- A: integer (nullable = true)
 |-- B: integer (nullable = true)
 |-- C: decimal(10,1) (nullable = true)
 |-- D: integer (nullable = true)
 |-- Spoiled: decimal(10,1) (nullable = true)



In [256]:
df_hack_data.show()

+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+
|Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|            Location|WPM_Typing_Speed|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+
|                   8.00|           391.09|              1|             2.96|           7.00|            Slovenia|           72.37|
|                  20.00|           720.99|              0|             3.04|           9.00|British Virgin Is...|           69.08|
|                  31.00|           356.32|              1|             3.71|           8.00|             Tokelau|           70.58|
|                   2.00|           228.08|              1|             2.48|           8.00|             Bolivia|           70.80|
|                  20.00|           408.50|              0|             3.57

In [257]:
df_hack_data.printSchema()

root
 |-- Session_Connection_Time: decimal(10,2) (nullable = true)
 |-- Bytes Transferred: decimal(10,2) (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: decimal(10,2) (nullable = true)
 |-- Pages_Corrupted: decimal(10,2) (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: decimal(10,2) (nullable = true)



##Random Forest

In [262]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [260]:
df_hack_data.show(10)

+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+
|Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|            Location|WPM_Typing_Speed|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+
|                   8.00|           391.09|              1|             2.96|           7.00|            Slovenia|           72.37|
|                  20.00|           720.99|              0|             3.04|           9.00|British Virgin Is...|           69.08|
|                  31.00|           356.32|              1|             3.71|           8.00|             Tokelau|           70.58|
|                   2.00|           228.08|              1|             2.48|           8.00|             Bolivia|           70.80|
|                  20.00|           408.50|              0|             3.57

In [259]:
df_hack_data.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [263]:
assembler = VectorAssembler(
    inputCols=['Session_Connection_Time',
               'Bytes Transferred',
               'Kali_Trace_Used',
               'Servers_Corrupted',
               'Pages_Corrupted',
               'WPM_Typing_Speed'],
               outputCol="features"
)

##Clustering

In [ ]:
root
 |-- Session_Connection_Time: decimal(10,2) (nullable = true)
 |-- Bytes Transferred: decimal(10,2) (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true) si estaba o no conectado
 |-- Servers_Corrupted: decimal(10,2) (nullable = true)
 |-- Pages_Corrupted: decimal(10,2) (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: decimal(10,2) (nullable = true)